In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly import graph_objs as go


from cities.utils.data_grabber import DataGrabber
from cities.queries.fips_query import FipsQuery

data = DataGrabber()


#### Use case: similarity in outcome patterns

You want to find top five juristiction with similar gdp change patterns. You value times nearest to you a bit more.

In [2]:
f  = FipsQuery(42001, "gdp", lag = 0, top =5, time_decay = 1.06)
f.find_euclidean_kins()
# you can inspect the weights resulting from your time_decay setting:
display(f.weigth_plot.show())

#you can find the distances and  inspect the resulting 
# dataframe that contains the ranking:

f.find_euclidean_kins()
#display(f.euclidean_kins)


None

### Use case: similarity in outcome patterns and some other features

Say you want to include historical population patterns in your similarity ranking. You also want to pay a bit more attention to older data points.

In [3]:
f  = FipsQuery(1007, outcome_var = "gdp",
               feature_groups= ["population"],
               weights= {"population":4}, #with one feature group only
               # weights 1-4 won't make a difference
               lag = 0, top =5, time_decay = 1.03)
f.find_euclidean_kins()
#display(f.weigth_plot.show())
#display(f.euclidean_kins)

#### Use case: similarity of outcome with a lag

You care about similarity of outcome variables, but your question now is: what other locations were 2 years ago in a similar place to me now, when it comes to the outcome variable and the features?


In [4]:
f  = FipsQuery(42001, "gdp", lag = 2, top =5, time_decay = 1.06)
f.find_euclidean_kins()

display(f.weigth_plot.show())

#f.find_euclidean_kins()
#display(f.euclidean_kins)

None

#### Use case: similarity of outcome and other features with a lag

In [5]:
f  = FipsQuery(20003, outcome_var = "gdp",
               feature_groups= ["population"],
               weights= {"population":4}, 
               lag = 2, top =5, time_decay = 1.03)
f.find_euclidean_kins()
display(f.weigth_plot.show())

None